<a href="https://colab.research.google.com/github/minawantinghsu/r4ds/blob/master/5400Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 24 kB/s 
     |████████████████████████████████| 198 kB 58.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=140a70020f30cd56089e2f6a28ef3e83b0f6d985700e7100fb34e0293b16a80b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
from pyspark.conf import SparkConf


In [4]:
from pyspark import SparkContext


In [5]:
from pyspark.sql import SQLContext



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

In [7]:
df = spark.read.csv('/content/drive/MyDrive/crunchbase_odm_orgs.csv', header=True)

In [8]:
df.show(10)

+--------------------+--------------------+------------+------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------+----------+------------+--------------------+
|                uuid|                name|        type|primary_role|              cb_url|       domain|        homepage_url|            logo_url|        facebook_url|         twitter_url|        linkedin_url|combined_stock_symbols|          city|    region|country_code|   short_description|
+--------------------+--------------------+------------+------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------+----------+------------+--------------------+
|e1393508-30ea-8a3...|            Wetpaint|organization|     company|https://www.crunc...| wetpaint.com|http://www.wetpai

In [9]:
#Q1: Find all entities with the name that starts with a letter "F" (e.g. Facebook, etc.):
#print the count and show() the resulting Spark DataFrame
df.filter(df.name.startswith("F")).show()



+--------------------+--------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+-------------+-------------+------------+--------------------+
|                uuid|                name|        type|primary_role|              cb_url|              domain|        homepage_url|            logo_url|        facebook_url|         twitter_url|        linkedin_url|combined_stock_symbols|         city|       region|country_code|   short_description|
+--------------------+--------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+-------------+-------------+------------+--------------------+
|df662812-7f97-0b4...|            Facebook|organization|     company|https://www.crunc...|    

In [10]:
df.filter(df.name.startswith("F")).count()

40601

In [ ]:
#Q2:Find all entities located in New York City:
#print the count and show() the resulting Spark DataFrame
df.filter(df.city == "New York").show()

+--------------------+--------------------+------------+------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------+--------+------------+--------------------+
|                uuid|                name|        type|primary_role|              cb_url|             domain|        homepage_url|            logo_url|        facebook_url|         twitter_url|        linkedin_url|combined_stock_symbols|    city|  region|country_code|   short_description|
+--------------------+--------------------+------------+------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------+--------+------------+--------------------+
|e1393508-30ea-8a3...|            Wetpaint|organization|     company|https://www.crunc...|       wetpaint.com|http://www.wetpai

In [11]:
df.filter(df.city == "New York").count()

27032

In [12]:
 import pyspark.pandas as ps

In [13]:
#Q3:Add a "Blog" column to the DataFrame with the row entries set to 1 
#if the "domain" field contains "blogspot.com", and 0 otherwise.
#show() only the records with the "Blog" field marked as 1

df.filter(df.domain.contains("blogspot.com")).count()



394

In [14]:
import pyspark.sql.functions as F

newdf = df.withColumn('Blog',F.when(df.domain.contains('blogspot.com') == 1,1).otherwise(0))

In [15]:
newdf.filter(newdf.Blog == 1).show()


+--------------------+--------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+-------------+------------+------------+--------------------+----+
|                uuid|                name|        type|primary_role|              cb_url|              domain|        homepage_url|            logo_url|        facebook_url|         twitter_url|        linkedin_url|combined_stock_symbols|         city|      region|country_code|   short_description|Blog|
+--------------------+--------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+-------------+------------+------------+--------------------+----+
|783b2aa6-7742-69e...|     Sad Urdu Poetry|organization|     company|https://www.c

In [ ]:
#Find all entities with names that are palindromes (name reads the same way forward and reverse, e.g. madam):
#print the count and show() the resulting Spark DataFrame 

In [43]:
def palindrome(row):
  return True if row[::-1].lower()==row.lower() else False

In [45]:
from pyspark.sql.functions import udf


In [35]:
import pyarrow

In [46]:
spark.conf.get("spark.sql.excution.arrow.enabled", "true")

'true'

In [47]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType

In [48]:
check_palindrome = udf(palindrome,BooleanType())

In [66]:
df2=df.select("*",check_palindrome("name"))

In [75]:
df2.filter(check_palindrome('name')).show()

+--------------------+-------+------------+------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+------------+--------------------+----------------+
|                uuid|   name|        type|primary_role|              cb_url|     domain|        homepage_url|            logo_url|        facebook_url|         twitter_url|        linkedin_url|combined_stock_symbols|                city|              region|country_code|   short_description|palindrome(name)|
+--------------------+-------+------------+------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+------------+--------------------+----------------+
|a94a240a-9929-424...| Revver|organization|     company|https://www

In [ ]:
df2.filter(check_palindrome('name')).count()